In [1]:
%matplotlib inline

In [2]:
import pandas as pd

In [3]:
import csv
import matplotlib.pyplot as plt

In [4]:
from scipy import stats

In [5]:
import math
import numpy as np

In [6]:
df_Casp = pd.read_csv("Casp8.txt",sep='\t');

/Users/tjyacoub/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
def unique(a, b):
    return list(set(a) != set(b))
def intersect(a, b):
    return list(set(a) & set(b))
def union(a, b):
    return list(set(a) | set(b))

In [20]:
def t_stat(df):
    
    names = df_to_names(df)
    n_samples1 = len(names.values()[0])
    n_samples2 = len(names.values()[1])
    
    index1 = 1
    index2 = index1 + n_samples1
    
    indices1 = [index1 + i for i in range(n_samples1)]
    indices2 = [index2 + i for i in range(n_samples2)]
    
    values1 = [df[df.columns[i]] for i in indices1]
    print sum(values1)
    values2 = [df[df.columns[i]] for i in indices2]
    
    mean1 = sum(values1)/n_samples1
    mean2 = sum(values2)/n_samples2
    
    var1_num = [(values1[i] - mean1)**2 for i in range(n_samples1)]
    var2_num = [(values2[i] - mean2)**2 for i in range(n_samples2)]
    
    var1 = sum(var1_num)/(n_samples1 - 1)
    var2 = sum(var2_num)/(n_samples2 - 1)
    
    t_stat = (mean1 - mean2) / (var1/n_samples1 + var2/n_samples2)**0.5
    pval = stats.t.sf(np.abs(t_stat), n_samples1-1)*2 
    
    logfc = np.log2(mean2/mean1)
    
    return [pval, logfc]

In [7]:
def df_to_names(df):
    
    name_dict = {}
        
    for i in range(1,len(df.columns)):
        name = df.columns[i][:-2]
        if name in name_dict:
            name_dict[name].append(i)
        else:
            name_dict[name] = [i]
            
    return name_dict

In [8]:
name_dict = df_to_names(df_Casp)

In [9]:
test_names = [name_dict.keys()[0], name_dict.keys()[1]]
test_names2 = [name_dict.keys()[2], name_dict.keys()[3]]

In [10]:
### Create new temp dataframe with cols ['samp1a','samp1b','samp2a','samp2b','gene']
### Add cols for logfc, pval
def names_to_df(names):
       
    # For first name    
    #print names
    indices = list(name_dict[names[0]])
    indices.insert(0,0)
    df_temp1 = df_Casp.ix[:,indices]
    
    # For second name
    indices = list(name_dict[names[1]])
    indices.insert(0,0)
    df_temp2 = df_Casp.ix[:,indices]
    
    dft = pd.merge(df_temp1, df_temp2, how='outer', on='Symbol')
    
    print "check"
    dft = dft.ix[1:];
    dft_float = dft.ix[:,1:].astype('float');
    dft_float = dft_float[dft_float > 0];
    dft['pval'] = t_stat(dft_float)[0];
    dft['logfc'] = t_stat(dft_float)[1];
    
    
    return dft
    #dft = merge(dft)

In [11]:
### Needs dataframe with cols ('gene', 'logfc', 'pval')

def filter_genes(df, pcut = 0.05, fcut = 1.4):
    
    df_pcut = df[df['pval'] < pcut]
    
    df_fc_up = df_pcut[df_pcut['logfc'] > fcut]
    
    df_fc_dn = df_pcut[df_pcut['logfc'] < -fcut]
    
    df_pv_up = df_pcut[df_pcut['logfc'] < fcut]
    df_pv_up = df_pv_up[df_pv_up['logfc'] > 0]
    
    df_pv_dn = df_pcut[df_pcut['logfc'] < 0]
    df_pv_dn = df_pv_dn[df_pv_dn['logfc'] > -fcut]

    fc_up_genes = df_fc_up['Symbol']
    fc_dn_genes = df_fc_dn['Symbol']
    pv_up_genes = df_pv_up['Symbol']
    pv_dn_genes = df_pv_dn['Symbol']
    
    #n_up_fc = sum(df_fc_up.ix[:,'up'])
    #n_dn_fc = sum(df_fc_dn.ix[:,'dn'])
    #n_up_pv = sum(df_pv_up.ix[:,'up'])
    #n_dn_pv = sum(df_pv_dn.ix[:,'dn'])

    return [fc_up_genes, fc_dn_genes, pv_up_genes, pv_dn_genes]


In [12]:

#### [state1, state2, state3, ..., stateN]
### [state1, state2, state3, ..., stateN]

### N-1 combinations for each state
### [state1-state2, state1-state3, state1-stateN ]

### N/2 * N combinations, *2 for reverse

### input dfs = [df1, df2]
def compare_edges(dfs):
    
    df1_genes = filter_genes(dfs[0])
    df2_genes = filter_genes(dfs[1])
    
    gene_totals = [[df1_genes[i], df2_genes[i]] for i in range(len(df1_genes))]
    gene_intersect = [intersect(i,j) for i,j in gene_totals]
    gene_percent = [float(len(gene_intersect[i]))/float(len(df1_genes[i])) for i in range(len(df1_genes))]
    ### What do we want?
    ### [[% up FC_1, % up FC_2],[% dn FC_1, %dn FC_1], [% up PV1, % up PV2], [% dn PV1, % dn PV2]]
    
    return gene_percent
    
    #n0 = len(dfres[k][0])
    #n1 = len(dfres[k][1])
    #ns = len(inter)
    
    #genes_shared_ct[k] = [n0, round(float(ns)/n0,3), n1, round(float(ns)/n1,3), ns]
    #gene_totals[k] = [len(dfres[k][0]),len(dfres[k][1])] 
                      #genes_unique[k] = unique(dfres[k][0],dfres[k][1])
    #gene_ratios[k] = len(genes_union

In [13]:
def combos(name_dict):
    names = name_dict.keys()
    n_names = len(names)
    stat_dict = {}
    
    #for i in range(len(names)):
    for i in range(n_names):
        name1 = names[i]
        stat_dict[name1] = {}
        #name_dict[names[i]]
        for j in range(i+1, 10): # n_names): #n_names):
            name2 = names[j]
            stat_dict[name1][name2] = {}
            dft1 = names_to_df([name1,name2])
            for k in range(j+1, 5): # n_names):
                name3 = names[k]
                stat_dict[name1][name2][name3] = {}
                
                for l in range(k+1, 6):#n_names):
                    name4 = names[l]
                    
                    print name1, name2, name3, name4
                    dft2 = names_to_df([name3, name4])
                    gene_perc = compare_edges([dft1,dft2])
                    if gene_perc[0] > 0:
                        stat_dict[name1][name2][name3][name4] = gene_perc
                    else:
                        stat_dict[name1][name2][name3][name4] = gene_perc
    return stat_dict
                        
                        
#        for j in range(i + 1, len(names)):

In [34]:
def combos2(name_dict):
    names = name_dict.keys()
    n_names = len(names)
    stat_dict = {}
    
    pairs = [[i, j] for i in range(n_names)]
    
    #for i in range(len(names)):
    for i in range(n_names):
        name1 = names[i]
        stat_dict[name1] = {}
        #name_dict[names[i]]
        
        for j in range(i+1, 8): # n_names): #n_names):
            name2 = names[j]
            stat_dict[name1][name2] = {}
            dft1 = names_to_df([name1,name2])
            
            for k in range(8, 15): # n_names):
                name3 = names[k]
                stat_dict[name1][name2][name3] = {}
                
                for l in range(15, 20):#n_names):
                    name4 = names[l]
                    
                    print name1, name2, name3, name4
                    dft2 = names_to_df([name3, name4])
                    gene_perc = compare_edges([dft1,dft2])
                    if gene_perc[0] > 0:
                        stat_dict[name1][name2][name3][name4] = gene_perc
                    else:
                        stat_dict[name1][name2][name3][name4] = gene_perc
    return stat_dict
                       

In [37]:
names = name_dict.keys()
n_names = len(names)

In [40]:
pairs = [[[names[i], names[j]] for i in range(n_names)] for j in range(n_names)]

In [42]:
for pair in pairs: 
    print pair

[['LysMC8flx.Sfhi.D20', 'LysMC8flx.Sfhi.D20'], ['LysMC8flx.IM.D20', 'LysMC8flx.Sfhi.D20'], ['LysMC8flx.mono.D14', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flxRIPK3.Sfhi.D14', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flxRIPK3.mono.D21', 'LysMC8flx.Sfhi.D20'], ['LysMC8flxRIPK3.Sflow.D14', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flx.Sflow.D20', 'LysMC8flx.Sfhi.D20'], ['LysMC8flxRIPK3.mono.D14', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flx.Sfhi.D0', 'LysMC8flx.Sfhi.D20'], ['LysMC8flxRIPK3C8flx.IM.D20', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flxRIPK3.Sfhi.D21', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flx.IM.D20', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flx.mono.D14', 'LysMC8flx.Sfhi.D20'], ['LysMC8flx.Sflow.D14', 'LysMC8flx.Sfhi.D20'], ['LysMC8flxRIPK3.IM.D14', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flx.Sfhi.D14', 'LysMC8flx.Sfhi.D20'], ['C8flx.mono.D14', 'LysMC8flx.Sfhi.D20'], ['CD11cC8flxRIPK3.mono.D14', 'LysMC8flx.Sfhi.D20'], ['LysMC8flx.Sfhi.D14', 'LysMC8flx.Sfhi.D20'], ['C8flx.Sflow.D14', 'LysMC8flx.Sfhi.D20'], ['LysMC8flx.IM.D14', 'LysMC8flx.Sfhi.D

In [35]:
%%capture
stat_dict = combos2(name_dict);

ZeroDivisionError: float division by zero

In [33]:
for name in stat_dict:
    if len(stat_dict[name]) > 0:
        print name, stat_dict[name], '\n'

LysMC8flx.Sfhi.D20 {'LysMC8flxRIPK3.mono.D14': {}, 'LysMC8flx.IM.D20': {'CD11cC8flxRIPK3.mono.D21': {'LysMC8flxRIPK3.Sflow.D14': [0.0, 0.0, 0.0, 0.0]}, 'LysMC8flx.mono.D14': {'CD11cC8flxRIPK3.mono.D21': [0.0, 0.0, 0.0011441647597254005, 0.0008361204013377926], 'LysMC8flxRIPK3.Sflow.D14': [0.027127003699136867, 0.0034542314335060447, 0.03318077803203661, 0.048494983277591976], 'CD11cC8flxRIPK3.Sfhi.D14': [0.02219482120838471, 0.03626943005181347, 0.03546910755148741, 0.08695652173913043]}, 'CD11cC8flxRIPK3.Sfhi.D14': {'CD11cC8flxRIPK3.mono.D21': [0.0, 0.0, 0.0, 0.0008361204013377926], 'LysMC8flxRIPK3.Sflow.D14': [0.0, 0.0, 0.043478260869565216, 0.01839464882943144]}}, 'LysMC8flx.mono.D14': {'CD11cC8flxRIPK3.mono.D21': {'LysMC8flxRIPK3.Sflow.D14': [0.0, 0.0, 0.0, 0.0]}, 'CD11cC8flxRIPK3.Sfhi.D14': {'CD11cC8flxRIPK3.mono.D21': [0.0, 0.0, 0.0, 0.0], 'LysMC8flxRIPK3.Sflow.D14': [0.003278688524590164, 0.0, 0.006261180679785331, 0.006896551724137931]}}, 'CD11cC8flxRIPK3.Sfhi.D14': {'CD11cC8fl

In [ ]:
names = ["FC up", "FC down", "PV up", "PV down"]
for i in range(4):
    print names[i]
    print x[i]
    #print genes_shared_ct[i] #".2f"% genes_shared_ct[i][1] 

In [ ]:
plt.plot([0.6, 0.2, 0.6, 0.5])

In [ ]:
dft = dft[pd.notnull(dft).all(axis=1)]

In [ ]:
ncols = len(dft.columns)
gene_names = dft.ix[:,0]
for i in range(1, ncols, 4):   #ncols):
    sample1_name = dft.columns[i]
    sample_list = []
    sample1 = dft.ix[:,i]
    for j in range(1, ncols):
        
        sample2_name = dft.columns[j]
        
        sample2 = dft.ix[:,j]
        #print sample1[0], sample2[0]
        logfc = np.log(sample2/sample1)
        logfc = logfc[logfc < 100]
        upreg = len(logfc[logfc > 1.4])
        downreg = len(logfc[logfc < -1.4])
        #print sample1_name, sample2_name, upreg, downreg
        dist = upreg + downreg
        regratio = upreg - downreg
        sample_list.append([sample2_name, dist, upreg, downreg])
        
    print sample1_name
    for n in sample_list:
        print n
    #print sample_list
#        print sample1/sample2

In [ ]:
def make_tree(dfs):
    ndf = len(dfs)
    mat
    for df1_ind in range(ndf):
        for df2_ind in range(df1_ind+1,ndf):
            merged_df = pd.merge(dfs[df1_ind], dfs[df2_ind], how = 'outer', on='gene')
            
            #merged_df['ratio'] = float(merged_df[[1]]) #sum(float(merged_df[[1,2,3,4]]))
            #for gene_ind in range(len(merged_df)):
        ### GET ratios
                
        ### GET SIMILARITY
        ### [ YN [1, 0.4, 0.8. 0.7],
        ###   ON [0.6, 1, 0.9, 0.2],
        ###   YF [0.4, 0.3, 1, 0.6],
        ###   OF [0.3, 0.9, 0.1, 1]]
        
        ### TREE FIGURE
        ###      YF (2)
        ###     /
        ###  YN(1)- ON(3)- OF(4)
        
        ###  [[1,2],[1,3],[3,4]]
        
        ### START WITH
        ### [[1,2],[1,3],[1,4]]
        ### [1-2] 0.3
        ### [1-3] 0.8
        ### [1-4] 0.5
        ### Then go to 2
        ### [2-1] 0.3 (ok)
        ### [2-3] 0.9 (>[1-3], change)
                    ### [[1,2],[2,3],[1,4]]
        ### [2-4] 0.2 (<[1-4], no change)
        ### On to 3...
        ### [3-1] 0.3 (ok)
        ### [3-2] 0.1 (<[1-3], no change)
        ### [3-4] 0.9 (>[1-4], change
                    ### [[1,2],[1,3],[3,4]])
        
        ### [1-3] ok
        sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
        for i in range(ndf):
            ## Initialize tree starting with i
            tree = []
            for j in range(ndf):
                tree[j] = [i,j]
                #similarity_matrix sim
                
                
            for j in range(ndf):
                sim_ij = sim[i,j]
                for k in range(ndf):
                    sim_ik = sim[i,k]
                    if sim_ik > sim_ij:
                        tree[j] = [i,k]
                    
                
                
    
    #return merged_df
        
            
make_tree(dfs);

In [ ]:
import random
ndf = 4
sim = [[random.random() for i in range(ndf)] for j in range(ndf)]

In [ ]:
sim[2][3]

In [ ]:
def make_tree2():
    ndf = 10
    sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
    nodes = [i for i in range(ndf)]
    for i in range(ndf):
        #print "begin", i
            ## Initialize tree starting with i
        tree = []# [0 for x in range(ndf-1)]
        
        j_nodes = [z for z in range(ndf)]
        j_nodes.remove(i)
        for j in j_nodes: 
            tree.append([i,j])
                #similarity_matrix sim
                
        n_iters = 1
        for iteration in range(n_iters):
            
            #print tree
        #print "j_nodes",j_nodes
            for j in j_nodes:
            #print tree
                sim_ij = sim[i][j]
            
                k_nodes = [z for z in range(ndf)]
                k_nodes.remove(j)
                k_nodes.remove(i)
                #print "k_nodes", k_nodes
                for k in k_nodes:
                #print "j = ", j, "k=", k
                    sim_ik = sim[i][k]
                    sim_jk = sim[j][k]
                    if sim_ik > sim_jk:
                        #print j,k, "> ", i, k
                        tree[k-1] = [j,k]
                        #print tree
            #print "end iteration", iteration, tree
            #for i in range(len(tree)):
            print sum([sim[ii][jj] for ii,jj in tree])
        printsorted(tree)
make_tree2()

In [ ]:
def make_tree_rand():
    ndf = 10
    sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
    seq = [i for i in range(ndf)]

    nodes = [random.choice() for i in range(ndf)]
    n_iter = 10
    
    ### Initialize tree
    tree = []# [0 for x in range(ndf-1)]
    
    j_nodes = [z for z in range(ndf)]
    j_nodes.remove(i)
    for j in j_nodes: 
        tree.append([i,j])
                #similarity_matrix sim
    for i in range(n_iter):
        #print "begin", i
            ## Initialize tree starting with i

                
        n_iters = 1
        for iteration in range(n_iters):
            
            #print tree
        #print "j_nodes",j_nodes
            for j in j_nodes:
            #print tree
                sim_ij = sim[i][j]
            
                k_nodes = [z for z in range(ndf)]
                k_nodes.remove(j)
                k_nodes.remove(i)
                #print "k_nodes", k_nodes
                for k in k_nodes:
                #print "j = ", j, "k=", k
                    sim_ik = sim[i][k]
                    sim_jk = sim[j][k]
                    if sim_ik > sim_jk:
                        #print j,k, "> ", i, k
                        tree[k-1] = [j,k]
                        #print tree
            #print "end iteration", iteration, tree
            #for i in range(len(tree)):
            error = sum([sim[ii][jj] for ii,jj in tree])
        printsorted(tree)
make_tree_rand()

In [ ]:
def make_tree_easy():
    ndf = 10
    sim = [[random.random() for i in range(ndf)] for j in range(ndf)]
    seq = [i for i in range(ndf)]

    nodes = [random.choice() for i in range(ndf)]
    n_iter = 10
    
    ### Initialize tree
    tree = []# [0 for x in range(ndf-1)]
    
    j_nodes = [z for z in range(ndf)]
    j_nodes.remove(i)
    for j in j_nodes: 
        tree.append([i,j])
                #similarity_matrix sim
    for i in range(n_iter):
        #print "begin", i
            ## Initialize tree starting with i

                
        n_iters = 1
        for iteration in range(n_iters):
            
            #print tree
        #print "j_nodes",j_nodes
            for j in j_nodes:
            #print tree
                sim_ij = sim[i][j]
            
                k_nodes = [z for z in range(ndf)]
                k_nodes.remove(j)
                k_nodes.remove(i)
                #print "k_nodes", k_nodes
                for k in k_nodes:
                #print "j = ", j, "k=", k
                    sim_ik = sim[i][k]
                    sim_jk = sim[j][k]
                    if sim_ik > sim_jk:
                        #print j,k, "> ", i, k
                        tree[k-1] = [j,k]
                        #print tree
            #print "end iteration", iteration, tree
            #for i in range(len(tree)):
            error = sum([sim[ii][jj] for ii,jj in tree])
        printsorted(tree)
make_tree_easy()

In [ ]:
tree = [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5]]

In [ ]:
def printsorted(tree):
    tree.sort()
    tree_dict = {}
    for i in range(len(tree)):
        if tree[i][0] not in tree_dict:
            tree_dict[tree[i][0]] = [tree[i][1]]
        else:
            tree_dict[tree[i][0]].append(tree[i][1])
        
    print tree_dict
            